In [1]:
import pandas as pd
import os
from typing import List, Sequence, Optional
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from typing import Mapping, Hashable, Optional, Iterable
from plotly.subplots import make_subplots
from statsmodels.formula.api import ols
import re
from backend.cuentas_nacionales.pib import PIBViews
from backend.cuentas_nacionales.pib.pib_constantes import ConstantesPIB as C
from backend.political_terms import tag_politics
from backend.plots import (plot_timeseries_bars, plot_bar_subplots_by_administration, 
                           simple_linegraph, plot_bars, create_pie_chart, 
                           plot_contributions)
from backend.utils import compute_basic_statistics, calculate_component_percentages, validate_percentage_sum, aggregate_columns
from backend.contributions import compute_contributions_pipeline, summary_contributions

In [2]:
g = lambda x: x.map(lambda x:x if type(x) == str else '{:,.2f}'.format(x))

year_on_year_changes = lambda df, q: (df.pct_change(q) * 100).dropna()

In [3]:
# pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [4]:
# colores por presidente:

presidential_colors = {
    "Calderón":   "#9ecae1",  
    "Olsen":      "#fb6a4a",  
    "Rodríguez":  "#fdd0a2",  
    "Pacheco":    "#9467bd",  
    "Arias":      "#74c476",  
    "Chinchilla": "#fdae6b",  
    "Solís":      "#c7c7c7",  
    "Alvarado":   "#e377c2",  
    "Chaves":     "#6baed6",  
}

# <font color = 'red'> 1. DATOS

Desde el punto de vista de la Oferta, la producción nacional se desagrega en distintas actividades económicas, tales como Agricultura, Comercio, Manufactura, Salud, Construcción, entre otras. En total, actualemente el BCCR calcula 15 actividades económicas.

Estas actividades pueden agruparse para analizar la producción según los sectores económicos: Primario, Secundario y Terciario.

- El sector primario está compuesto por la actividad agrícola.

- El sector secundario corresponde a la industria, que incluye las actividades de Manufactura, Electricidad, Minas y Construcción.

- El sector terciario agrupa los servicios, que comprenden el resto de las actividades económicas.

En síntesis, desde la perspectiva de la Oferta, la producción nacional se puede analizar en dos niveles: por actividades económicas y por sectores.

Asimismo, todas estas variables pueden analizarse desde diversas perspectivas analíticas, cada una de las cuales ofrece una visión complementaria del desempeño económico:

- Crecimiento económico: mide la variación porcentual interanual de la producción, permitiendo identificar períodos de expansión o contracción en las distintas actividades y sectores. Ejemplo: en el mes $X$ del año $t$ la actividad agrícola creció 3% interanualmente.

- Composición: muestra la estructura relativa de la producción, es decir, el peso o participación que tiene cada actividad o sector dentro del total de la economía. Ejemplo: en el mes $X$ del año $t$ la actividad agrícola pesó un 5% de la producción total.

- Contribución al crecimiento económico: cuantifica el aporte específico de cada actividad o sector al crecimiento total, combinando tanto su tasa de crecimiento como su peso dentro de la producción nacional. Ejemplo: en el mes $X$ del año $t$ la ecnomomía total creció un 6%, de los cuales la actividad agrícola aportó un punto porcentual.

En conjunto, estos enfoques permiten comprender no solo cuánto crece la economía, sino también qué sectores impulsan dicho crecimiento y cómo evoluciona la estructura productiva del país.

## <font color = 'red'> 1.1 DATOS EN MILLONES DE COLONES

In [5]:
folder = r'C:\Users\adolj\OneDrive\Documentos\APPS\informed_economist\data\raw'
file_name = 'Variables_PIB_TC.xlsx'
input_path = os.path.join(folder, file_name)
dic_0 = pd.read_excel(input_path, sheet_name = None)

# datos en niveles (millones de colones)
quarterly_data_levels_TC = dic_0['pibQ'].copy()
quarterly_data_levels_TC.set_index('fecha', inplace = True)

display(g(quarterly_data_levels_TC))

,PIB_TC,PIB_Impuestos_TC,PIB_Valor_Agregado_TC,PIB_Agricultura_Silvicultura_Pesca_TC,PIB_Minas_Canteras_TC,PIB_Manufactura_TC,PIB_Electricidad_Agua_Saneamiento_TC,PIB_Construccion_TC,PIB_Comercio_TC,PIB_Transporte_Almacenamiento_TC,PIB_Hoteles_Restaurantes_TC,PIB_Informacion_Comunicaciones_TC,PIB_Financieras_Seguros_TC,PIB_Inmobiliario_TC,PIB_Actividades_Profesionales_TC,PIB_Administracion_Publica_TC,PIB_Ense_Salud_Asistencia_Social_TC,PIB_Otras_Actividades_TC,PIB_Demanda_Interna_TC,PIB_Gasto_Consumo_Final_TC,PIB_Gasto_Consumo_Final_Hogares_TC,PIB_Bienes_Consumo_Duradero_TC,PIB_Bienes_Consumo_Semi_Duraderos_TC,PIB_Bienes_Consumo_No_Duraderos_TC,PIB_Servicios_TC,PIB_Gasto_Consumo_Final_Gobierno_General_TC,PIB_Formacion_Bruta_Capital_Fijo_TC,PIB_Maquinaria_Equipo_TC,PIB_Nuevas_Construcciones_TC,PIB_Exportaciones_Bienes_Servicios_TC,PIB_Exportaciones_Bienes_TC,PIB_Exportaciones_Servicios_TC,PIB_Exportacion_Bienes_RegDef_TC,PIB_Exportacion_Bienes_RegEsp_TC,PIB_Importaciones_Bienes_Servicios_TC,PIB_Importaciones_Bienes_TC,PIB_Importaciones_Servicios_TC,PIB_Importacion_Bienes_RegDef_TC,PIB_Importacion_Bienes_RegEsp_TC,PIB_Combustibles_TC,PIB_Sin_Combustibles_TC,PIB_RegDef_TC,PIB_RegEsp_TC,PIB_Terminos_Intercambio,PIB_USD
fecha,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1991-03-31,"2,635,525.91","172,871.28","2,470,959.47","186,009.27","14,918.22","515,443.43","72,725.12","114,554.65","324,540.28","169,865.27","79,141.25","14,630.09","55,303.37","123,725.39","90,559.28","244,197.15","514,589.86","120,268.95","2,653,741.80","2,320,227.25","1,694,421.77","40,932.12","104,839.15","798,564.72","820,576.91","710,228.34","346,160.83","195,059.14","142,387.49","561,613.95","382,929.05","145,551.42","350,452.96","66,616.35","564,484.52","446,849.79","132,397.02","358,298.64","89,788.06","123,911.05","268,101.87","2,559,981.55","53,619.16",80.28,"1,934.18"
1991-06-30,"2,673,951.20","177,989.72","2,497,945.50","183,121.65","15,430.58","521,062.46","74,734.70","119,324.57","331,482.03","170,831.88","80,520.43","15,091.59","52,667.51","125,559.39","91,638.25","244,999.80","518,322.49","120,485.48","2,694,327.37","2,358,367.89","1,724,633.51","42,565.38","106,259.89","821,621.74","820,860.43","713,842.81","345,285.88","184,993.60","150,624.12","587,868.68","400,695.74","157,383.78","364,100.02","71,602.13","596,913.10","462,832.11","139,948.32","373,470.13","92,592.96","129,701.93","281,519.51","2,594,045.03","57,427.80",90.63,"1,740.47"
1991-09-30,"2,720,923.67","183,543.11","2,538,092.95","186,599.39","15,737.90","534,680.98","76,206.66","124,466.12","336,841.98","172,483.95","82,382.04","15,303.68","50,218.13","128,137.12","92,372.10","245,953.11","522,556.37","120,583.63","2,739,337.68","2,383,063.23","1,750,696.67","44,206.29","106,902.11","838,229.36","820,044.36","717,430.89","350,159.25","181,988.38","158,111.11","614,073.20","420,675.34","169,953.43","377,074.81","77,608.25","625,689.64","473,749.80","148,865.70","383,200.77","95,258.14","134,286.29","293,415.65","2,636,617.32","60,523.26",89.14,"1,709.24"
1991-12-31,"2,784,461.85","191,589.16","2,597,364.74","192,894.17","16,174.38","557,179.55","77,842.14","130,267.86","344,422.47","174,581.29","85,044.15","15,441.94","49,590.92","131,785.37","94,097.92","247,318.13","527,864.39","120,771.24","2,805,220.41","2,423,230.05","1,785,784.73","47,125.11","108,151.98","856,195.79","830,239.55","721,755.55","370,006.09","193,861.17","165,658.28","643,008.87","443,742.11","180,708.89","390,768.12","85,034.92","654,803.47","481,192.53","158,404.55","388,365.91","97,808.42","137,884.00","300,405.72","2,696,004.45","63,766.12",98.14,"1,831.83"
1992-03-31,"2,852,134.91","200,263.32","2,656,834.65","195,747.29","16,974.81","571,167.12","79,005.31","136,015.51","354,386.10","176,738.22","88,229.29","15,646.97","50,924.16","136,779.71","97,265.82","249,022.11","534,663.31","121,341.44","2,884,762.50","2,481,857.52","1,830,903.88","51,253.10","111,159.20","878,351.56","851,094.62","727,473.74","399,1

## <font color = 'red'> 1.2 CLASE PIBViews

In [6]:
# Esta clase permite acceder de una manera ordenada a las diferentes desagregaciones de Oferta y Demanda.

# Se instancia pasándole a su constructor los datos en millones de colones:

ob_pibv = PIBViews(quarterly_data_levels_TC)

# Luego se acceden a desagregaciones predefinidas. Ejemplo, aceder a los sectores primario, terciario y secundario:

sectores = ob_pibv.oferta.get_sectores()

# O bien, a las actividades económicas:

actividades_economicas = ob_pibv.oferta.get_industrias()



## <font color = 'red'> 1.3 Administraciones

In [7]:
# La función tag_politics permite asignarle a cada mes-año un label la administración correspondiente:

# Ejemplo: 

sectores = ob_pibv.oferta.get_sectores()

tagged_df = tag_politics(sectores)

g(tagged_df)

,Agro_TC,Servicios_TC,Industria_Ampliada_TC,President,Party,Term,Label
fecha,,,,,,,
1991-03-31,"186,009.27","1,736,820.90","717,641.42",Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1991-06-30,"183,121.65","1,751,598.85","730,552.31",Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1991-09-30,"186,599.39","1,766,832.12","751,091.67",Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1991-12-31,"192,894.17","1,790,917.82","781,463.93",Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-03-31,"195,747.29","1,824,997.14","803,162.75",Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
...,...,...,...,...,...,...,...
2024-09-30,"432,703.55","7,238,061.72","2,245,270.20",Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2024-12-31,"426,246.96","7,292,230.77","2,286,739.47",Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2025-03-31,"420,410.15","7,360,881.17","2,336,887.26",Rodrigo Chaves Robles,PPSD,2022–2026,Chaves


In [8]:
tagged_df.groupby('Label').count()['President']

Label
Alvarado      16
Arias         16
Calderón      13
Chaves        14
Chinchilla    16
Olsen         16
Pacheco       16
Rodríguez     16
Solís         16
Name: President, dtype: int64

# <font color = 'red'> 2. RESULTADOS

## <font color = 'red'> 2.1 TASAS DE CRECIMIENTO INTERANUALES

Se mide el ritmo de crecimiento de la producción de bienes y servicios. 

Para cada variable $Y_t$ se calcula su tasa de crecimiento interanual: $\Delta \% = \frac{Y_t}{Y_{t-12}} - 1$ 

### <font color = 'red'> 2.1.1 ECONOMÍA NACIONAL

In [9]:
# Notar el flujo:
#   con el objeto PIBViews se accede a la información requerida. Luego se le calculan los cambios de crecimiento interanuales:

df = year_on_year_changes(ob_pibv.oferta.get_categorias_principales(), q = 4)
df.columns = df.columns.str.replace('_TC', '')
df.info()
print('\n')
col = 'PIB'
statistic = 'mean'

pib = df[[col]]
pib = tag_politics(pib)
pib = pib[pib['Label'] != 'Calderón']

stats_by_admin = compute_basic_statistics(pib, value_col=col, groupby_col='Label')
stats_by_admin = stats_by_admin.sort_values(by = statistic, ascending = False).round(2)

fig_pib_label = plot_timeseries_bars(
    df=pib,
    start = '1994-06-30',
    y_col=col,
    category_col="Label",
    title=f"Costa Rica: Crecimiento Interanual del {col} por Administración",
    x_title="",
    y_title="(%)",
    y_round=2,
    y_suffix="",
    width=850,
    height=420,
    zero_line=False,
    showlegend=True,
    colors=presidential_colors,
    template='plotly_white',
    show_y_grid=True
)

fig_pib_label.show()

display(stats_by_admin[[statistic]].transpose())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 135 entries, 1992-03-31 to 2025-09-30
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PIB                 135 non-null    float64
 1   PIB_Valor_Agregado  135 non-null    float64
 2   PIB_Impuestos       135 non-null    float64
dtypes: float64(3)
memory usage: 4.2 KB




Label,Arias,Chaves,Rodríguez,Pacheco,Chinchilla,Olsen,Solís,Alvarado
mean,4.8,4.51,4.34,4.25,4.18,4.12,3.88,2.35


### <font color = 'red'> 2.1.2 PIB POR ACTIVIDAD ECONÓMICA

In [10]:
df = year_on_year_changes(ob_pibv.oferta.get_industrias(), q = 4)
df.columns = df.columns.str.replace('_TC', '')
df.info()
print('\n')
col = 'PIB_Construccion'
statistic = 'mean'

pib = df[[col]]
pib = tag_politics(pib)
pib = pib[pib['Label'] != 'Calderón']

stats_by_admin = compute_basic_statistics(pib, value_col=col, groupby_col='Label')
stats_by_admin = stats_by_admin.sort_values(by = statistic, ascending = False).round(2)

fig_pib_label = plot_timeseries_bars(
    df=pib,
    start = '1994-06-30',
    y_col=col,
    category_col="Label",
    title=f"Costa Rica: Crecimiento Interanual {col} por Administración",
    x_title="",
    y_title="(%)",
    y_round=2,
    y_suffix="",
    width=850,
    height=420,
    zero_line=False,
    showlegend=True,
    colors=presidential_colors,
    template='plotly_white',
    show_y_grid=True
)

fig_pib_label.show()

display(stats_by_admin[[statistic]].transpose())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 135 entries, 1992-03-31 to 2025-09-30
Data columns (total 15 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   PIB_Agricultura_Silvicultura_Pesca  135 non-null    float64
 1   PIB_Minas_Canteras                  135 non-null    float64
 2   PIB_Manufactura                     135 non-null    float64
 3   PIB_Electricidad_Agua_Saneamiento   135 non-null    float64
 4   PIB_Construccion                    135 non-null    float64
 5   PIB_Comercio                        135 non-null    float64
 6   PIB_Transporte_Almacenamiento       135 non-null    float64
 7   PIB_Hoteles_Restaurantes            135 non-null    float64
 8   PIB_Informacion_Comunicaciones      135 non-null    float64
 9   PIB_Financieras_Seguros             135 non-null    float64
 10  PIB_Inmobiliario                    135 non-null    float64
 11  PIB_Actividades_Profesiona

Label,Arias,Rodríguez,Pacheco,Chaves,Olsen,Solís,Chinchilla,Alvarado
mean,9.75,8.75,3.95,2.76,2.52,2.15,-2.0,-2.47


### <font color = 'red'> 2.1.3 POR SECTORES (PRIMARIO, SECUNDARIO, TERCIARIO)

In [11]:
df = ob_pibv.oferta.get_sectores()
df.columns = df.columns.str.replace('_TC', '')
df.rename(columns={'Industria_Ampliada': 'Industria'}, inplace = True)
df = year_on_year_changes(df, q = 4)
df = tag_politics(df).drop(['President', 'Party', 'Term'], axis = 1)


components = ['Agro', 'Servicios', 'Industria']
means = df.groupby("Label")[components].mean()
to_remove = ["Calderón", "Pacheco", "Rodríguez", "Olsen"]
means = means.drop(index=to_remove, errors="ignore")
fig = plot_bar_subplots_by_administration(df = means, variables = components, colors = presidential_colors, title = "PIB: Promedio por Administración de los Componentes de la Demanda", y_title = "Crecimiento Interanual (%)")
fig.show()

g(means.loc[['Chaves', 'Alvarado', 'Solís', 'Chinchilla', 'Arias']])

,Agro,Servicios,Industria
Label,,,
Chaves,0.75,4.43,5.52
Alvarado,0.22,1.90,4.00
Solís,2.19,4.58,2.11
Chinchilla,2.72,5.20,1.21
Arias,1.77,6.04,2.35


### <font color = 'red'> 2. COMPOSICIÓN

In [12]:
# Esta perspectiva de análisis nos dice cuánto pesa una variable con respecto a un total. Es na simple división.

# Por ejemplo:

#   ¿Cuánto pesa la actividad agrícola dentro del total de la economía a una fecha dada?
#   o bien, ¿cómo ha evolucionado ese peso relativo a lo largo del tiempo?

#   ¿Cuánto pesa el sector terciario? ¿cómo ha evolucionado dicho peso?

#### <font color = 'red'> 2.1 PESO RELATIVO EN UNA FECHA DADA

In [13]:
# Se calcula cuánto pesa cada industria dentro del total de la economía:

df_weights = calculate_component_percentages(ob_pibv.oferta.get_industrias())

In [14]:
# Son muchas actividades económicas, entonces si se quiere visualizar todas en un mismo gráfico se puede hacer algo así:

fig = plot_bars(
    df_weights.loc["2025-09-30"],
    orientation="h",
    color_mode="single",
    bar_color="#185ed7",
    sort="asc",
    bar_opacity= 0.40,
    top_n=None,
    decimals=1,
    label_suffix="%",
    title="Peso Relativo de Cada Actividad Económica dentro del PIB",
    subtitle="",
    xaxis_title="%",
    yaxis_title="Actividad Económica"
)
fig.show()

# La gráfica nos indica que a setiembre de 2025, en CR, la actividad económica más importante (con mayor peso relativo dentro del PIB) es
# la Manufactura. Mientras que la menos importante es Minas y Canteras.


In [15]:
# Lo anterior también lo podemos ver en un gráfico pastel, pero como son tantas actividades económicas, 
# entonces podemos agrupar las más pequeñas, antes de calcular los porcentajes:

# Datos en millones de colones de las 15 actividades económicas:
df_industrias = ob_pibv.oferta.get_industrias()

# Agregamos las industrias que a la fecha de corte pesan menos que un 5% de la economía:

df = aggregate_columns(df = df_industrias,
                       cols_to_aggregate=['PIB_Agricultura_Silvicultura_Pesca_TC',
                                          'PIB_Construccion_TC', 
                                          'PIB_Administracion_Publica_TC', 
                                          'PIB_Otras_Actividades_TC', 
                                          'PIB_Hoteles_Restaurantes_TC',
                                          'PIB_Electricidad_Agua_Saneamiento_TC',
                                          'PIB_Minas_Canteras_TC'],
                       new_col_name="Otras",
                       return_mode="others_plus_new"
)

# Calculamos los pesos relativos:

df_weights = calculate_component_percentages(df)

# Y graficamos un pastel:

filtered = df_weights.loc['2025-09-30'] # Seleccionamos una fecha específica:
df = pd.DataFrame(filtered.values, filtered.index, columns=['Peso Relativo']).reset_index(names=['Actividad'])
create_pie_chart(df, 'Actividad', 'Peso Relativo', title_text = 'Costa Rica: Peso Relativo de las Actividades Económicas', subtitle_text = '2025-09-30')

In [16]:
# Otro caso: ¿cuánto pesan los sectores de la economía?:

df_weights = calculate_component_percentages(ob_pibv.oferta.get_sectores())

fig = plot_bars(
    df_weights.loc["2025-09-30"],
    orientation="h",
    color_mode="single",
    bar_color="#185ed7",
    sort="asc",
    bar_opacity= 0.40,
    top_n=None,
    decimals=1,
    label_suffix="%",
    title="Peso Relativo de los Sectores Primario, Secundario y Terciario dentro del PIB",
    subtitle="",
    xaxis_title="%",
    yaxis_title="Sector Económico"
)
fig.show()

# La gráfica nos indica que a setiembre de 2025, CR es una economía de servicios:

In [17]:
# Cómo son pocas sub-variables, queda más bonito un pastel:

df_weights = calculate_component_percentages(ob_pibv.oferta.get_sectores())

# Y graficamos un pastel:

filtered = df_weights.loc['2025-09-30'] # Seleccionamos una fecha específica:
df = pd.DataFrame(filtered.values, filtered.index, columns=['Peso Relativo']).reset_index(names=['Actividad'])
create_pie_chart(df, 'Actividad', 'Peso Relativo', title_text = 'Costa Rica: Peso Relativo de Los Sectores Económicos', subtitle_text = '2025-09-30')

#### <font color = 'red'> 2.2 EVOLUCIÓN DEL PESO RELATIVO

In [18]:
# En vez de prguntarnos solo por cuánto pesa una variable en una fecha dada, podemos preguntarnos cómo ha evolucionado a lo largo
# del tiempo dicha métrica.

In [19]:
# Por ejemplo, a continuación se muestra la evolución del peso relativo de las actividades económicas a lo largo de la muestra.
# Como son muchas actividades económicas se muestran en tres gráficos:

# Obtenemos los datos:
df_weights = calculate_component_percentages(ob_pibv.oferta.get_industrias())

# Graficamos:
s = df_weights.iloc[-1]
s = s[s > 10]
fig = simple_linegraph(df = df_weights[s.index], 
                 y_columns = s.index, 
                 title='Evolución del Peso Relativo de las Actividades con Peso Superior al 10% del PIB', 
                 title_color="gray", xaxis_title='', yaxis_title='%', 
                 width=800, height=400, decimals=2, secondary_y='Servicios_TC', secondary_y_title='', 
                 line_colors=None, yaxis_title_color="gray", secondary_yaxis_title_color="gray")

fig.show()


s = df_weights.iloc[-1]
s = s[(s > 5) & (s < 10)]
fig = simple_linegraph(df = df_weights[s.index], 
                 y_columns = s.index, 
                 title='Evolución del Peso Relativo de las Actividades con Peso entre 5% y 10% del PIB', 
                 title_color="gray", xaxis_title='', yaxis_title='%', 
                 width=800, height=400, decimals=2, secondary_y='Servicios_TC', secondary_y_title='', 
                 line_colors=None, yaxis_title_color="gray", secondary_yaxis_title_color="gray")

fig.show()

s = df_weights.iloc[-1]
s = s[(s < 5)]
fig = simple_linegraph(df = df_weights[s.index], 
                 y_columns = s.index, 
                 title='Evolución del Peso Relativo de las Actividades con Peso Inferior al 5%', 
                 title_color="gray", xaxis_title='', yaxis_title='%', 
                 width=800, height=400, decimals=2, secondary_y='Servicios_TC', secondary_y_title='', 
                 line_colors=None, yaxis_title_color="gray", secondary_yaxis_title_color="gray")

fig.show()


In [20]:
# De manera similar, podemos ver la evoluvión del peso relativo de los sectores Primario, Terciario y Secundario:

# Se observa el cambio estructural de la economía desde 2006, de una economía industrial (manufacturera) a una de servicios:

df_weights = calculate_component_percentages(ob_pibv.oferta.get_sectores())

simple_linegraph(df = df_weights, 
                 y_columns = ['Agro_TC',	'Servicios_TC', 'Industria_Ampliada_TC'], 
                 title='Evolución de la estructura del PIB por Sector Primario, Secundario y Terciario', 
                 title_color="gray", xaxis_title='', yaxis_title='Agro e Industria', 
                 width=800, height=400, decimals=2, secondary_y='Servicios_TC', secondary_y_title='', 
                 line_colors=None, yaxis_title_color="gray", secondary_yaxis_title_color="gray")

### <font color = 'red'> 3. CONTRIBUCIONES

In [21]:
# Suponga que el crecimiento interanual de la economía como un todo fue de 5%.
# Entonces cabe preguntarse ¿cuánto contribuyó cada uno de sus componentes?
# Por ejemplo, ¿cuánto contribuyó el agro, la manufactura, etc?
# La suma de las contribuciones debería dar el 5% en cuestión.

# En general, el cálculo de las contribuciones requiere de dos insumos:
# Supongamos que tenemos una variable Y que se desagrega en tres variables, es decir, Y = Z1 + Z2 + Z3. Supongamos que Y creció 
# interanualmente un 5%. Entonces, para calcular las contribuciones, se requiere de dos insumos:
#   - los pesos relativos de Z1, Z2, Z3, y 
#   - las tasas de crecimiento interanuales de Y, Z1, Z2 y Z3.
# La suma de las contribuciones de Z1, Z2, y Z3 debe ser, en este ejemplo, de 5%.

In [22]:
# Ejemplo 1: voy a calcular la contribución que cada atividad económica tuvo en el crecimiento total:

# 1) Cálculo de pesos relativos: ¿cuánto pesa cada actividad económica del total?
# Esto ya lo calculamos más arriba:
df_weights = calculate_component_percentages(ob_pibv.oferta.get_industrias())

# 2) Tasas de crecimiento interanuales:
df_growth = year_on_year_changes(ob_pibv.oferta.get_industrias(), q = 4)

# -------> Cálculo de contribuciones:

contrib_pp, weights_validation, tidy_panel = compute_contributions_pipeline(df_growth=df_growth, 
                                                                            df_weights=df_weights,
                                                                              return_long  = True)

# contrib_pp contiene las contribuciones calculadas para todos los meses de la muestra.

# Luego lo grafico.

In [23]:
# Ejemplo 2: voy a calcular la contribución que cada atividad económica tuvo en el crecimiento total:

# 1) Cálculo de pesos relativos: ¿cuánto pesa cada actividad económica del total?
# Esto ya lo calculamos más arriba:
df_weights = calculate_component_percentages(ob_pibv.oferta.get_sectores())

# 2) Tasas de crecimiento interanuales:
df_growth = year_on_year_changes(ob_pibv.oferta.get_sectores(), q = 4)

# -------> Cálculo de contribuciones:

contrib_pp, weights_validation, tidy_panel = compute_contributions_pipeline(df_growth=df_growth, 
                                                                            df_weights=df_weights,
                                                                              return_long  = True)

contrib_pp.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 127 entries, 1994-03-31 to 2025-09-30
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Agro_TC                127 non-null    float64
 1   Industria_Ampliada_TC  127 non-null    float64
 2   Servicios_TC           127 non-null    float64
dtypes: float64(3)
memory usage: 4.0 KB


In [24]:
fig_sector = plot_contributions(
    df=contrib_pp,
    cols=["Agro_TC", "Industria_Ampliada_TC", "Servicios_TC"],
    title="Contribución al Crecimiento Interanual por Sector Primario, Secundario y Terciario",
    yaxis_title="Contribuciones (p.p.) y Crecimiento Interanual (%)",
    total_mode="sum",
    colors = {"Agro_TC":"orangered",              
              "Industria_Ampliada_TC": "steelblue",  
              "Servicios_TC": "gold"          
    },
    total_color= "black"
)

fig_sector.show()